### MID 5
### Taking ant1.3 as training data.
### ant1.4 as testing data.
### Min max scaling done to few columns: ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
### Oversampling and smote methods used to increase size of training data.
### CNN model used, model type is 2 as per the BTP documentation spreadsheet.
### SVD with n_components = 15
### np.rint() used on predictions

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Dense,Dropout,Conv2D,Conv1D,Flatten,MaxPool2D
import tensorflow as tf
import csv
from sklearn.decomposition import TruncatedSVD

In [2]:
# Loading data. 
train_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv"
test_data_path = "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"
train_data_name = "ant-1.3"
test_data_name = "ant-1.4"
ant_1_3 = pd.read_csv(train_data_path)
ant_1_4 = pd.read_csv(test_data_path)
files = ["/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.3.csv", "/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/datasets/ant-1.4.csv"]
combined_data = pd.concat(map(pd.read_csv, files))

In [3]:
# Applying Min Max Scaling.
scaler = MinMaxScaler()
MinMaxScaler(copy=True, feature_range=(0, 1))
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
combined_data[cols_to_norm] = MinMaxScaler().fit_transform(combined_data[cols_to_norm])

In [4]:
combined_data

name  version                                             name.1  \
0    ant      1.3            org.apache.tools.ant.taskdefs.ExecuteOn   
1    ant      1.3                 org.apache.tools.ant.DefaultLogger   
2    ant      1.3     org.apache.tools.ant.taskdefs.TaskOutputStream   
3    ant      1.3                  org.apache.tools.ant.taskdefs.Cvs   
4    ant      1.3             org.apache.tools.ant.taskdefs.Copyfile   
..   ...      ...                                                ...   
173  ant      1.4                   org.apache.tools.ant.TaskAdapter   
174  ant      1.4  org.apache.tools.ant.taskdefs.rmic.DefaultRmic...   
175  ant      1.4           org.apache.tools.ant.IntrospectionHelper   
176  ant      1.4  org.apache.tools.ant.taskdefs.compilers.Defaul...   
177  ant      1.4                org.apache.tools.ant.NoBannerLogger   

          wmc  dit    noc       cbo       rfc      lcom        ca  ...  \
0    0.142857  0.6  0.050  0.102941  0.214286  0.011856  0.014815  ...   
1    0.181818  0.0  0.025  0.058824  0.163265  0.020033  0.029630  ...   
2    0.038961  0.2  0.000  0.007353  0.045918  0.000000  0.000000  ...   
3    0.155844  0.4  0.000  0.088235  0.188776  0.013083  0.000000  ...   
4    0.077922  0.4  0.000  0.029412  0.107143  0.000409  0.000000  ...   
..        ...  ...    ...       ...       ...       ...       ...  ...   
173  0.064935  0.4  0.000  0.051471  0.096939  0.001635  0.029630  ...   
174  0.220779  0.0  0.075  0.110294  0.336735  0.036795  0.029630  ...   
175  0.298701  0.0  0.000  0.227941  0.418367  0.082175  0.162963  ...   
176  0.142857  0.0  0.200  0.161765  0.438776  0.006132  0.059259  ...   
177  0.051948  0.2  0.000  0.022059  0.081633  0.000000  0.000000  ...   

          dam       moa       mfa       cam    ic       cbm       amc  \
0    1.000000  0.111111  0.885057  0.232323  0.75  0.363636  0.165951   
1    1.000000  0.000000  0.000000  0.307692  0.00  0.000000  0.080979   
2    1.000000  0.111111  0.714286  0.666667  0.25  0.090909  0.083267   
3    1.000000  0.111111  0.770833  0.458333  0.00  0.000000  0.115693   
4    1.000000  0.000000  0.880952  0.416667  0.50  0.181818  0.100881   
..        ...       ...       ...       ...   ...       ...       ...   
173  0.000000  0.000000  0.902439  0.400000  0.25  0.090909  0.138351   
174  1.000000  0.222222  0.000000  0.197917  0.00  0.000000  0.172373   
175  0.444444  0.000000  0.000000  0.318182  0.00  0.000000  0.219724   
176  1.000000  0.777778  0.000000  0.266667  0.00  0.000000  0.373390   
177  1.000000  0.000000  0.842105  0.875000  0.25  0.090909  0.064852   

       max_cc    avg_cc  bug  
0    0.085714  0.209085    0  
1    0.171429  0.269903    2  
2    0.028571  0.109529    0  
3    0.085714  0.232742    0  
4    0.028571  0.136898    0  
..        ...       ...  ...  
173  0.028571  0.131428    0  
174  0.400000  0.396221    0  
175  0.685714  0.378561    0  
176  0.628571  0.761689    1  
177  0.142857  0.287498    0  

[303 rows x 24 columns]

In [7]:
# define transform
components = 15
svd = TruncatedSVD(n_components=components)
# prepare transform on dataset
svd.fit(combined_data[cols_to_norm])
# apply transform to dataset
transformed = svd.transform(combined_data[cols_to_norm])

In [8]:
transformed = pd.DataFrame(transformed)
transformed 

0         1         2         3         4         5         6  \
0    1.697270 -0.240716  0.571264  0.167873 -0.202270 -0.188764 -0.085932   
1    1.016611 -0.267720 -0.587890 -0.054598 -0.014350 -0.077750 -0.160155   
2    1.298451 -0.173527  0.139837 -0.541003 -0.007964  0.011583  0.156168   
3    1.537891 -0.056428  0.028502 -0.104266  0.280907  0.130204 -0.176106   
4    1.530257 -0.074334  0.374525 -0.205480 -0.061914 -0.152907 -0.063597   
..        ...       ...       ...       ...       ...       ...       ...   
298  0.886501  0.524176  0.551332  0.103135  0.083555  0.172359  0.095620   
299  1.128822 -0.426056 -0.629426  0.258850 -0.034627  0.177394 -0.220035   
300  1.027511 -0.126484 -0.591917  0.769665  0.027848  0.483424  0.049615   
301  1.318002 -0.514244 -0.775534  0.478137 -0.191742  0.531927 -0.176518   
302  1.534499  0.032362  0.068811 -0.514331 -0.062047  0.142505  0.205071   

            7         8         9        10        11        12        13  \
0    0.036019 -0.075786  0.129869 -0.074938 -0.025339 -0.066285  0.001433   
1    0.052980  0.100634 -0.030916 -0.066994 -0.048365 -0.007133  0.003084   
2   -0.110641 -0.028186  0.008754  0.139273 -0.107446 -0.018334  0.006414   
3   -0.029262 -0.088801 -0.025663 -0.118818  0.003933  0.018829 -0.043154   
4    0.021766  0.089365  0.037752  0.031946 -0.114260 -0.091288 -0.014637   
..        ...       ...       ...       ...       ...       ...       ...   
298  0.012480  0.004141  0.017085  0.037964 -0.029034 -0.045604 -0.017821   
299  0.025682 -0.128906  0.061409 -0.099610 -0.022340  0.050520  0.067642   
300 -0.134999 -0.142810  0.233455 -0.574456 -0.054281  0.160251  0.095574   
301  0.103397 -0.589727  0.040915  0.212815  0.047857  0.022727  0.095875   
302 -0.085574  0.014607 -0.033795  0.029711 -0.200737 -0.019112  0.003207   

           14  
0   -0.015661  
1    0.027329  
2   -0.066103  
3    0.048522  
4   -0.040312  
..        ...  
298  0.022297  
299 -0.067599  
300 -0.128321  
301 -0.128328  
302 -0.035288  

[303 rows x 15 columns]

In [9]:
train_data_index_list = list(range(ant_1_3.shape[0]))
train_data_index_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124]

In [10]:
test_data_index_list = list(range(ant_1_3.shape[0], ant_1_3.shape[0] + ant_1_4.shape[0]))
test_data_index_list

[125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291

In [11]:
X_train = transformed[transformed.index.isin(train_data_index_list)]
X_train = np.array(X_train)
Y_train = combined_data[transformed.index.isin(train_data_index_list)]
Y_train = Y_train['bug']

X_test = transformed[transformed.index.isin(test_data_index_list)]
X_test = np.array(X_test)
Y_test = combined_data[transformed.index.isin(test_data_index_list)]
Y_test = Y_test['bug']

In [13]:
# Applying over sampling and SMOTE to training data for augmentation.
ros = RandomOverSampler(random_state=0)
X_train, Y_train = ros.fit_resample(X_train, Y_train)

smt = SMOTE()
X_train, Y_train = smt.fit_resample(X_train, Y_train)


In [15]:
#Creating X_train dataframe after SMOTE
X_train = pd.DataFrame(X_train)
X_train

0         1         2         3         4         5         6  \
0    1.697270 -0.240716  0.571264  0.167873 -0.202270 -0.188764 -0.085932   
1    1.016611 -0.267720 -0.587890 -0.054598 -0.014350 -0.077750 -0.160155   
2    1.298451 -0.173527  0.139837 -0.541003 -0.007964  0.011583  0.156168   
3    1.537891 -0.056428  0.028502 -0.104266  0.280907  0.130204 -0.176106   
4    1.530257 -0.074334  0.374525 -0.205480 -0.061914 -0.152907 -0.063597   
..        ...       ...       ...       ...       ...       ...       ...   
415  1.759963 -0.333453  0.451183  0.469646 -0.100127 -0.107335 -0.084072   
416  1.725241 -0.085638  0.404703 -0.022151 -0.112624  0.057648 -0.065983   
417  1.014200 -0.374213 -0.547216  0.119112  0.129458 -0.113918 -0.205231   
418  1.014200 -0.374213 -0.547216  0.119112  0.129458 -0.113918 -0.205231   
419  1.538579 -0.225167  0.445009 -0.090284 -0.104056 -0.107362 -0.032935   

            7         8         9        10        11        12        13  \
0    0.036019 -0.075786  0.129869 -0.074938 -0.025339 -0.066285  0.001433   
1    0.052980  0.100634 -0.030916 -0.066994 -0.048365 -0.007133  0.003084   
2   -0.110641 -0.028186  0.008754  0.139273 -0.107446 -0.018334  0.006414   
3   -0.029262 -0.088801 -0.025663 -0.118818  0.003933  0.018829 -0.043154   
4    0.021766  0.089365  0.037752  0.031946 -0.114260 -0.091288 -0.014637   
..        ...       ...       ...       ...       ...       ...       ...   
415 -0.119410  0.187599  0.245221 -0.084559  0.137752 -0.074704  0.087766   
416  0.008807  0.121036  0.121905  0.038621  0.129920 -0.101186  0.004151   
417  0.006024  0.047777  0.153280 -0.022765  0.060597  0.030842 -0.044536   
418  0.006024  0.047777  0.153280 -0.022765  0.060597  0.030842 -0.044536   
419  0.014913 -0.051659  0.111349  0.032772 -0.115809  0.060401 -0.056393   

           14  
0   -0.015661  
1    0.027329  
2   -0.066103  
3    0.048522  
4   -0.040312  
..        ...  
415 -0.031954  
416  0.023943  
417 -0.009617  
418 -0.009617  
419 -0.001124  

[420 rows x 15 columns]

In [16]:
#Creating Y_train dataframe after SMOTE
Y_train = pd.DataFrame(Y_train, columns=['bug'])
Y_train

bug
0      0
1      2
2      0
3      0
4      0
..   ...
415    3
416    3
417    3
418    3
419    3

[420 rows x 1 columns]

In [17]:
#Creating X_test dataframe after SMOTE
X_test = pd.DataFrame(X_test)
X_test 

0         1         2         3         4         5         6  \
0    1.523293 -0.170090  0.128456 -0.001951  0.090125  0.031779 -0.137793   
1    1.651678 -0.063255  0.368565 -0.141057 -0.087231 -0.068837  0.000961   
2    1.022280 -0.147043 -0.594706 -0.317492  0.019030 -0.195015  0.039704   
3    1.737926 -0.093301  0.515939 -0.033409 -0.255796 -0.140888  0.030092   
4    1.080908 -0.466366 -0.483825  0.268978 -0.007205 -0.222265 -0.038584   
..        ...       ...       ...       ...       ...       ...       ...   
173  0.886501  0.524176  0.551332  0.103135  0.083555  0.172359  0.095620   
174  1.128822 -0.426056 -0.629426  0.258850 -0.034627  0.177394 -0.220035   
175  1.027511 -0.126484 -0.591917  0.769665  0.027848  0.483424  0.049615   
176  1.318002 -0.514244 -0.775534  0.478137 -0.191742  0.531927 -0.176518   
177  1.534499  0.032362  0.068811 -0.514331 -0.062047  0.142505  0.205071   

            7         8         9        10        11        12        13  \
0   -0.000681 -0.080393 -0.047305  0.049114 -0.014516 -0.034881 -0.028537   
1   -0.087265  0.030013  0.213726 -0.015526 -0.052851  0.002593 -0.026785   
2   -0.070796  0.063123 -0.009621 -0.037356 -0.000902  0.006305  0.017992   
3    0.008599  0.055056  0.027625 -0.059313 -0.033507 -0.131585  0.015888   
4   -0.104178  0.324192 -0.044102  0.065395 -0.068383 -0.052320  0.030716   
..        ...       ...       ...       ...       ...       ...       ...   
173  0.012480  0.004141  0.017085  0.037964 -0.029034 -0.045604 -0.017821   
174  0.025682 -0.128906  0.061409 -0.099610 -0.022340  0.050520  0.067642   
175 -0.134999 -0.142810  0.233455 -0.574456 -0.054281  0.160251  0.095574   
176  0.103397 -0.589727  0.040915  0.212815  0.047857  0.022727  0.095875   
177 -0.085574  0.014607 -0.033795  0.029711 -0.200737 -0.019112  0.003207   

           14  
0   -0.027541  
1    0.016895  
2    0.026292  
3   -0.066634  
4    0.064540  
..        ...  
173  0.022297  
174 -0.067599  
175 -0.128321  
176 -0.128328  
177 -0.035288  

[178 rows x 15 columns]

In [18]:
#Creating Y_test dataframe after SMOTE
Y_test = pd.DataFrame(Y_test, columns=['bug'])
Y_test

bug
0      0
1      0
2      0
3      0
4      0
..   ...
173    0
174    0
175    0
176    1
177    0

[178 rows x 1 columns]

In [19]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,components

In [20]:
# Preparing the data for the model.
X_train_matrix = X_train.values
X_test_matrix = X_test.values
Y_train_matrix = Y_train.values
Y_test_matrix = Y_test.values

In [21]:
# Preparing the data for the model.
Ytrainseries = Y_train['bug']
Ytestseries = Y_test['bug']

In [22]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [23]:
#Building the model
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=1, activation='relu',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=1, activation='relu'))
model.add(Conv2D(16, kernel_size=1, activation='relu'))
    
    
#model.add(MaxPool2D(pool_size=(1,8)))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='relu'))

#compile model using mse as the loss function
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 1, 15, 64)         128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 15, 32)         2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 15, 16)         528       
_________________________________________________________________
flatten (Flatten)            (None, 240)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1928      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 4,673
Trainable params: 4,673
Non-trainable params: 0
______________________________________________________

In [25]:
# Fitting the model on training data.
history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 128)

Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 2/100
4/4 [==============================] - 0s 3ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 4/100
4/4 [==============================] - 0s 3ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 5/100
4/4 [==============================] - 0s 3ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - m

4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 47/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 48/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 49/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 50/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 51/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5

4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 92/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 93/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 94/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 95/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5000 - root_mean_squared_error: 1.8708 - mean_squared_logarithmic_error: 0.9023
Epoch 96/100
4/4 [==============================] - 0s 2ms/step - loss: 3.5000 - mse: 3.5000 - mae: 1.5

In [26]:
# Getting the predictions.
predictions_y = model.predict(X_test1)
predictions_y

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.   

In [27]:
# Rounding off the predictions to nearest
#integer as count of bugs is an integer.
predictions_y_round = np.rint(predictions_y)
predictions_y_round 

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [28]:
# Viewing the number of predictions.
predictions_y_round.shape

(178, 1)

In [29]:
# Getting the sum of all the predictions obtained to used while obtaining FPA
s = 0
for  t in range(predictions_y_round.shape[0]):
    s+=predictions_y_round[t]
s

array([0.], dtype=float32)

In [30]:
# Obtaining the value of FPA metric for the model
Fpa = 0
for  t in range(predictions_y_round.shape[0]):
        x = 0
        for j in range( predictions_y_round.shape[0]-t+1, predictions_y_round.shape[0]):
               x = x + predictions_y_round[j]
        
        x = (x/s)/predictions_y_round.shape[0]
        Fpa = Fpa + x
Fpa

<ipython-input-30-aead2945b2b8>:8: RuntimeWarning: invalid value encountered in true_divide
  x = (x/s)/predictions_y_round.shape[0]


array([nan], dtype=float32)

In [31]:
# Obtaining the value of CLC metric for the model
previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s

CLC = 0
for i in range(predictions_y_round.shape[0]):
    if(i==0):
        CLC += 0 + previous_obtained
    else:
        additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s
        CLC += 2*previous_obtained + additional
        previous_obtained += additional
        
CLC/=(2*predictions_y_round.shape[0])
CLC

<ipython-input-31-6dbac094958d>:2: RuntimeWarning: invalid value encountered in true_divide
  previous_obtained = predictions_y_round[predictions_y_round.shape[0] - 1]/s
<ipython-input-31-6dbac094958d>:9: RuntimeWarning: invalid value encountered in true_divide
  additional = (predictions_y_round[predictions_y_round.shape[0] - 1 - i])/s


array([nan], dtype=float32)

In [32]:
# Getting direct metric results using the metrics given to model.
score = model.evaluate(X_test1, Y_test_matrix)
print(history.history.keys())
model.test_on_batch(X_test1, Y_test_matrix)
model.metrics_names
#print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[2], score[2]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[3], score[3]*100))


6/6 [==============================] - 0s 658us/step - loss: 0.3652 - mse: 0.3652 - mae: 0.2641 - root_mean_squared_error: 0.6043 - mean_squared_logarithmic_error: 0.1364
dict_keys(['loss', 'mse', 'mae', 'root_mean_squared_error', 'mean_squared_logarithmic_error'])


['loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error']

In [33]:
# Saving the model
model_id = 5
path_to_save = '/home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_2'
model.save(path_to_save)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /home/bavanya/Desktop/6thSem/BTP/regression_PROMISE_dataset/saved_models/ant1.3_ant1.4_model2/ant1.3_ant1.4_model2_2/assets


In [34]:
# Writing the results to csv file.
heading = ['model_id', 'train_data_name', 'test_data_name'] + model.metrics_names + ['fpa', 'clc']
score = [model_id, train_data_name, test_data_name] + score + [float(Fpa) , float(CLC)]

In [35]:
heading

['model_id',
 'train_data_name',
 'test_data_name',
 'loss',
 'mse',
 'mae',
 'root_mean_squared_error',
 'mean_squared_logarithmic_error',
 'fpa',
 'clc']

In [36]:
score

[5,
 'ant-1.3',
 'ant-1.4',
 0.36516904830932617,
 0.36516904830932617,
 0.26409927010536194,
 0.6042922139167786,
 0.1364068239927292,
 nan,
 nan]

In [37]:
# Writing the results to csv file.
with open(path_to_save + '_metric_results.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(heading)
    writer.writerow(score)


In [38]:
# Loading the model and to make sure that the model is saved properly.
model_loaded = load_model(path_to_save)